In [1]:
library(MutationalPatterns)
library(mSigAct)
library(tidyverse)
library(ApobeX)
library(ggplot2)

Lade n"otiges Paket: GenomicRanges

Lade n"otiges Paket: stats4

Lade n"otiges Paket: BiocGenerics


Attache Paket: 'BiocGenerics'


Die folgenden Objekte sind maskiert von 'package:stats':

    IQR, mad, sd, var, xtabs


Die folgenden Objekte sind maskiert von 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Lade n"otiges Paket: S4Vectors


Attache Paket: 'S4Vectors'


Die folgenden Objekte sind maskiert von 'package:base':

    I, expand.grid, unname


Lade n"otiges Paket: IRanges

Lade n"otiges Paket: GenomeInfoDb

Lade n"otiges Paket: NMF

Lade n"otiges Paket: registry

Lade n"otiges Paket: rngtools

Lade n"otiges Paket

In [2]:
signatures = get_known_signatures()

apo_sigs = signatures[,c(2,19)]
prior_knowledge_subset = signatures[,c(1,2,5,6,19,21,25)]

In [3]:
#import bootstrapped background corrected matrix
all = read.csv('../../data/mutation_tables/all_corrected.csv', row.names = 1)
all = as.matrix(all)

In [4]:
#get samples
# order of genotypes = control, HUWE1, UBR4, UBR5
samples_d10 = c('S110', 'S123', 'S68',
                'S77', 'S135', 'S142',
                 'S52', 'S62', 'S37',
                 'S64', 'S56', 'S41'
                 )

mut_mat = subset_matrix_by_samples(all, samples_d10)

#reorder matrix so that genotype order is control, UBR4, UBR5, HUWE1
order = c('S110', 'S123', 'S68',
                 'S52', 'S62', 'S37',
                 'S64', 'S56', 'S41',
                 'S77', 'S135', 'S142'
                 )

mut_mat = mut_mat[, match(order, colnames(mut_mat))]

In [5]:
#adjust rownames for mSigAct
#read the vector from the file
row_names_m_sig_format <- scan("../utility/row_names_msigAct.txt", what = character(), sep = "\n")

rownames(mut_mat) = row_names_m_sig_format

#adjust rownames on signature catalog and sigs of interest
rownames(apo_sigs) = row_names_m_sig_format #sigs_interest
rownames(prior_knowledge_subset) = row_names_m_sig_format # 

In [6]:
#perform signature presence test for SBS2 and SBS13
all.signature.precense.tests = list()

for (sig in colnames(apo_sigs)) {
  all.signature.precense.tests[[sig]] = 
    SignaturePresenceTest(
      spectra = mut_mat,
      sigs = prior_knowledge_subset,
      target.sig.index = sig,
      m.opts = DefaultManyOpts(),
      seed = 1234,
      mc.cores = 3
    )
}

In [7]:
#create summary statistics
presence.test.summary.stats = list()

for (sig in names(all.signature.precense.tests) ) {
  
  df = data.frame( 
    #pull out likelihood ratio test statistic and p_value
    statistic = sapply(all.signature.precense.tests[[sig]],
                       function(x) x$statistic),
    chisq.p = sapply(all.signature.precense.tests[[sig]],
                     function(x) x$chisq.p) 
  )

  #threshold p_value --> anything greater than 0.05 set to 1
  # because -log(1) = 0 for when I plot -logp
  df$chisq.p[ df$chisq.p > 0.05] = 1
  df$minus_logP = -log10(df$chisq.p)
  df = df %>% rownames_to_column(var = "Sample")

  #add test condition annotation
  condition = c(rep('control', 3), rep('UBR4', 3), rep('UBR5', 3), rep('HUWE1', 3))
  df$condition = condition

  #add signature annotation
  df$signature = c(rep(sig, 12))

  # Log Transformation
  df$log_statistic <- log(df$statistic + 1)  # Adding 1 to avoid logarithm of zero
  #df$log_statistic= df$log_statistic+1

  # Standardization
  # mean_statistic <- mean(df$statistic)
  # sd_statistic <- sd(df$statistic)
  # df$standardized_statistic <- (df$statistic - mean_statistic) / sd_statistic

  # # Normalization
  # min_statistic <- min(df$statistic)
  # max_statistic <- max(df$statistic)
  # df$normalized_statistic <- (df$statistic - min_statistic) / (max_statistic - min_statistic)
  df = tibble(df)
  presence.test.summary.stats[[sig]] = df
}

In [8]:
presence.test.summary.stats

Sample,statistic,chisq.p,minus_logP,condition,signature,log_statistic
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
S110,3.021642e-03,1.000000e+00,0.000000,control,SBS2,3.017086e-03
S123,5.944178e-04,1.000000e+00,0.000000,control,SBS2,5.942412e-04
S68,0.000000e+00,1.000000e+00,0.000000,control,SBS2,0.000000e+00
S52,1.455192e-11,1.000000e+00,0.000000,UBR4,SBS2,1.455192e-11
S62,3.917086e+00,4.779767e-02,1.320593,UBR4,SBS2,1.592716e+00
S37,6.716644e+01,2.495258e-16,15.602885,UBR4,SBS2,4.221952e+00
S64,1.080321e+02,2.644556e-25,24.577647,UBR5,SBS2,4.691643e+00
S56,1.405926e+01,1.771397e-04,3.751684,UBR5,SBS2,2.711993e+00
S41,0.000000e+00,1.000000e+00,0.000000,UBR5,SBS2,0.000000e+00


In [9]:
#group summary stats by genotype for individual plots

df_SBS2 = presence.test.summary.stats$SBS2
df_SBS13 = presence.test.summary.stats$SBS13

grouped_condition_summary_stats = list()

types = c('control', 'UBR4', 'UBR5', 'HUWE1')

for (type in types){
    
    df2 = subset(df_SBS2, condition == type)
    df13 = subset(df_SBS13, condition == type)

    df = rbind(df2, df13)

    grouped_condition_summary_stats [[type]] = df
}

grouped_condition_summary_stats 

Sample,statistic,chisq.p,minus_logP,condition,signature,log_statistic
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
S110,3.021642e-03,1,0,control,SBS2,3.017086e-03
S123,5.944178e-04,1,0,control,SBS2,5.942412e-04
S68,0.000000e+00,1,0,control,SBS2,0.000000e+00
S110,1.738922e-01,1,0,control,SBS13,1.603249e-01
S123,-6.411106e-06,1,0,control,SBS13,-6.411127e-06
S68,5.848051e-10,1,0,control,SBS13,5.848051e-10
Sample,statistic,chisq.p,minus_logP,condition,signature,log_statistic
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
S52,1.455192e-11,1.000000e+00,0.000000,UBR4,SBS2,1.455192e-11


In [10]:
#plotting summary plots by signature
custom_palette_2 = c("#077E97", '#AB242C', '#19874B', "#767B7B")
custom_palette = c('#0066aa', '#0066aa', '#0066aa', '#0066aa')

local.fig.dir = '../plots'

for (sig in names(presence.test.summary.stats) ) {
    sig.presence.p = presence.test.summary.stats[[sig]] %>% 
    mutate(Sample = factor(Sample,levels = Sample)) %>% 
    ggplot(aes(x = Sample, y = log_statistic, alpha = minus_logP, fill = condition)) + 
    geom_bar(stat = "identity") +
    scale_fill_manual(values= custom_palette)+
    # facet_wrap(~Sig, nrow = 3, scales = "free_y") + 
    theme_bw(base_size = 10) + 
    # theme(axis.text.x = element_text(angle = 90, vjust = 0.5, size = 9)) 
        theme(axis.text.x = element_text(angle = 0, size = 8),
          panel.grid = element_blank(),
           plot.title = element_text(size = 12, hjust = 0.5)) + 
    scale_x_discrete(labels = presence.test.summary.stats[[sig]]$Sample) + 
    ggtitle(paste0("Signature presence test for ", sig))+
    geom_vline(xintercept = seq(3.5, 12, by = 3), linetype = "dashed")+
        ylab("log(likelihood ratio)")
  ggsave(plot = sig.presence.p, filename = file.path(local.fig.dir, paste0(sig, ".pdf")),
        width = 5.3, height = 4)
}

In [11]:
#combine signature SBS2 and SBS13 into the same plot
sig_combined = rbind(presence.test.summary.stats$SBS2, presence.test.summary.stats$SBS13)
sig_combined

Sample,statistic,chisq.p,minus_logP,condition,signature,log_statistic
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
S110,3.021642e-03,1.000000e+00,0.000000,control,SBS2,3.017086e-03
S123,5.944178e-04,1.000000e+00,0.000000,control,SBS2,5.942412e-04
S68,0.000000e+00,1.000000e+00,0.000000,control,SBS2,0.000000e+00
S52,1.455192e-11,1.000000e+00,0.000000,UBR4,SBS2,1.455192e-11
S62,3.917086e+00,4.779767e-02,1.320593,UBR4,SBS2,1.592716e+00
S37,6.716644e+01,2.495258e-16,15.602885,UBR4,SBS2,4.221952e+00
S64,1.080321e+02,2.644556e-25,24.577647,UBR5,SBS2,4.691643e+00
S56,1.405926e+01,1.771397e-04,3.751684,UBR5,SBS2,2.711993e+00
S41,0.000000e+00,1.000000e+00,0.000000,UBR5,SBS2,0.000000e+00


In [12]:
# Define the custom palettes
custom_palette_a <- c('#0066aa', '#AB242C')
custom_palette_b <- c('#0066aa', '#19874B')

# Ensure 'Sample' column is a factor with unique levels
sig_combined$Sample <- factor(sig_combined$Sample, levels = unique(sig_combined$Sample))

# Create the plot
sig_combined_plot <- ggplot(sig_combined, aes(x = Sample, y = log_statistic, alpha = minus_logP, fill = signature)) + 
  geom_bar(stat = "identity", position = "dodge") +
  scale_fill_manual(values = custom_palette_b) +
  theme_bw(base_size = 10) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, size = 9)) +
  ylab("log(likelihood ratio)") +
  ggtitle("Signature presence test for APOBEC related signatures") +
  geom_vline(xintercept = seq(3.5, 12, by = 3), linetype = "dashed")

# Define the local figure directory
local.fig.dir = '../plots'

# Create the directory if it does not exist
if (!dir.exists(local.fig.dir)) {
  dir.create(local.fig.dir, recursive = TRUE)
}

# Save the plot
ggsave(plot = sig_combined_plot, filename = file.path(local.fig.dir, "APOBEC_combined_msigact.pdf"), width = 5.3, height = 4)
